## House Price Prediction with MlFlow

- Run a hyperparameter tuning while training a model.
- Log every hyperparameter and metric.
- Compare the results of the various runs in the MlFlow UI.
- Choose the best run and register it as a model.

In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn as sklearn
from mlflow.models import infer_signature
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing


In [2]:
housing = fetch_california_housing()
housing_df = pd.DataFrame(housing.data, columns=housing.feature_names)
housing_df['Price'] = housing.target
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
from urllib.parse import urlparse

# Split data
X = housing_df.drop(columns=['Price'])
y = housing_df['Price']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get signature
signature = infer_signature(X_train, y_train)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

In [ ]:
# Hyperparameter tuning using GridSearchCV
def hyperparameter_tuning(x_train, y_train):
    """
    Perform hyperparameter tuning using GridSearchCV.
    :param x_train:
    :param y_train:
    :return: grid_search object
    """
    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(random_state=42),
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=3,
        verbose=2,
        n_jobs=-1
    )
    grid_search.fit(x_train, y_train)
    return grid_search

"Fitting 3 folds for each of 24 candidates, totalling 72 fits"

This message is generated by GridSearchCV from scikit-learn during hyperparameter tuning. Here's what it means:

3 folds: The data is being split into 3 parts (folds) for cross-validation. Each fold is used as a validation set once, while the remaining folds are used for training.

24 candidates: The hyperparameter grid contains 24 combinations of hyperparameter values to test. This is determined by the param_grid you defined.

72 fits: Since there are 24 combinations of hyperparameters and 3 folds for cross-validation, the total number of model fits is ( 24 \times 3 = 72 ).

This process ensures that each hyperparameter combination is evaluated thoroughly using cross-validation.

In [7]:
mlflow.set_experiment("House Pricing")

with mlflow.start_run():
    # Perform hyperparameter tuning
    grid_search = hyperparameter_tuning(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Log parameters and metrics
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_score", -best_score)
    mlflow.log_metric("mse", mse)

    # Tracking url
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Log the model
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="house_price_model",
            signature=signature,
            input_example=X_train[:5],                     # Optional: Example input for the model
            registered_model_name="House_Price_Predictor"  # Optional: Name for the registered model
        )
    else:
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="house_price_model",
            signature=signature,
            input_example=X_train[:5],  # Optional: Example input for the model
        )

2025/04/28 08:38:50 INFO mlflow.tracking.fluent: Experiment with name 'House Pricing' does not exist. Creating a new experiment.


Fitting 3 folds for each of 24 candidates, totalling 72 fits


Successfully registered model 'House_Price_Predictor'.
2025/04/28 08:40:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: House_Price_Predictor, version 1


🏃 View run funny-mare-675 at: http://127.0.0.1:5000/#/experiments/586088074569693845/runs/ceb6d869c102460a8721c972e659ebbd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/586088074569693845


Created version '1' of model 'House_Price_Predictor'.
